In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
IMAGE_SIZE = 224 # Image size of resize when applying transforms.
BATCH_SIZE = 64
NUM_WORKERS = 4 # Number of parallel processes for data preparation.
SUBMISSION = 'submission.csv'
DATA_DIR = 'test'
MODEL_DIR = 'model.pth'

In [4]:
import torchvision.models as models
import torch.nn as nn

def build_model(pretrained=True, fine_tune=True, num_classes=11):
    if pretrained:
        print('[INFO]: Loading pre-trained weights')
    else:
        print('[INFO]: Not loading pre-trained weights')
    model = models.efficientnet_b0(pretrained=pretrained)
    if fine_tune:
        print('[INFO]: Fine-tuning all layers...')
        for params in model.parameters():
            params.requires_grad = True
    elif not fine_tune:
        print('[INFO]: Freezing hidden layers...')
        for params in model.parameters():
            params.requires_grad = False
            
    num_features = model.classifier[-1].in_features
    # Change the final classification head.
    model.classifier[-1] = nn.Sequential(
      nn.Dropout(p=0.5),
      nn.Linear(num_features,num_classes),
      nn.Softmax(dim=1) 
    )
    return model

In [5]:
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
            )
])

In [6]:
import os
from PIL import Image
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = os.listdir(root_dir)
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        img_name = self.image_names[idx]
        uid = int(img_name[len("test"):img_name.find(".")])
#         uid = int(img_name[:img_name.find(".")])
        return image, uid

In [7]:
dataset_test = CustomDataset(
    DATA_DIR, 
    transform=test_transform
)

In [8]:
test_loader = DataLoader(
        dataset_test, batch_size=BATCH_SIZE, 
        shuffle=False
)

In [9]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [10]:
model = build_model().to(device)

checkpoint = torch.load(MODEL_DIR, map_location=device)
print('Loading trained model weights...')
model.load_state_dict(checkpoint)

[INFO]: Loading pre-trained weights
[INFO]: Fine-tuning all layers...
Loading trained model weights...


C:\Users\OSM\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\OSM\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [11]:
import pandas as pd

In [12]:
ids = list()
classes = list()
threshold = 0.5

with torch.no_grad():
    for ( inputs, labels ) in tqdm(test_loader):
        model.eval()
        
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        scores, predicted = torch.max(outputs.data, 1)
        
        for i in range(len(scores)):
            if scores[i].item() < threshold:
                print(f"Image {labels[i].item()} is unclassified with score:", scores[i].item())
                predicted[i] = 8 # assign spam class to the prediction
            
        classes.extend(predicted.cpu().numpy())
        ids.extend(labels.cpu().numpy())

should_be = []
for i in range(len(ids)):
    target = 1
    # anything in `test` should not be spam  except ids > 10000
    if ids[i] > 10000 :
        target = 0
        
    if classes[i] == 8 :
        classes[i] = 0
    else :
        classes[i] = 1
    should_be.append(target)

        
df = pd.DataFrame({'ID': ids, 'class': classes})
df.to_csv(SUBMISSION, index=False)

  0%|          | 0/74 [00:00<?, ?it/s]

Image 10012 is unclassified with score: 0.3290358781814575
Image 10024 is unclassified with score: 0.47946202754974365
Image 1023 is unclassified with score: 0.4992971122264862
Image 1035 is unclassified with score: 0.4001902639865875
Image 1137 is unclassified with score: 0.443750262260437
Image 1152 is unclassified with score: 0.4669169783592224
Image 1190 is unclassified with score: 0.33299022912979126
Image 1217 is unclassified with score: 0.4083831012248993
Image 1259 is unclassified with score: 0.44746100902557373
Image 1329 is unclassified with score: 0.45257261395454407
Image 1335 is unclassified with score: 0.4619213938713074
Image 1370 is unclassified with score: 0.41854190826416016
Image 1469 is unclassified with score: 0.45848095417022705
Image 1515 is unclassified with score: 0.39781078696250916
Image 1548 is unclassified with score: 0.40578746795654297
Image 1556 is unclassified with score: 0.40597692131996155
Image 1660 is unclassified with score: 0.43530645966529846
Ima

In [13]:
from sklearn.metrics import confusion_matrix
# Calculate confusion matrix
cm = confusion_matrix(should_be, classes)

print(cm)

[[  22    5]
 [ 165 4483]]
